In [1]:
import os
from pathlib import Path
from glob import glob

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import RocCurveDisplay, auc
from scipy.stats import rankdata


In [2]:
PATH_ROOT = Path(os.getcwd()).absolute().parent.parent
print(PATH_ROOT)

/home/lukec/workspace/applicabilityDomain


In [3]:
CLF_alias = ['KNN', 'NN', 'RF', 'SVM']
AD_alias = ['DM-δ',
            'DM-γ',
            'DM-κ',
            'BB',
            'Prob.',
            'Magnet',
            'FS',
            'RC'
]
AL = ['Magnet', 'FS', 'RC']
DATASETS = ['Ames', 'BBBP', 'Cancer', 'CYP1A2', 'hERG', 'HIV', 'Liver']

In [4]:
path_aut = os.path.join(PATH_ROOT, 'plot_results', 'roc', 'auc.csv')
df_auc = pd.read_csv(path_aut)

df_auc

,Dataset,Classifier,Method,AUC,Rank
0,Ames,KNN,DM-δ,0.586356,7
1,Ames,KNN,DM-γ,0.601921,5
2,Ames,KNN,DM-κ,0.587643,6
3,Ames,KNN,BB,0.528490,8
4,Ames,KNN,Prob.,0.705766,2
...,...,...,...,...,...
219,Liver,SVM,BB,0.480144,8
220,Liver,SVM,Prob.,0.654888,1
221,Liver,SVM,Magnet,0.644450,2
222,Liver,SVM,FS,0.634226,3


In [5]:
df_auc_pivot = df_auc.pivot(index=['Dataset', 'Classifier'], columns='Method',  values='Rank')
# Rearrange columns
df_auc_pivot = df_auc_pivot[AD_alias]

df_auc_pivot

Method              DM-δ  DM-γ  DM-κ  BB  Prob.  Magnet  FS  RC
Dataset Classifier                                             
Ames    KNN            7     5     6   8      2       4   1   3
        NN             6     5     7   8      2       4   1   3
        RF             8     5     6   7      1       4   3   2
        SVM            7     5     6   8      2       4   1   3
BBBP    KNN            5     6     7   8      3       4   2   1
        NN             3     5     6   7      2       4   1   8
        RF             4     7     6   8      1       5   2   3
        SVM            5     6     7   8      3       4   1   2
CYP1A2  KNN            5     6     7   8      1       4   2   3
        NN             5     6     7   8      1       4   2   3
        RF             5     6     7   8      1       4   3   2
        SVM            5     6     7   8      1       4   3   2
Cancer  KNN            7     5     6   8      3       4   1   2
        NN             7     6     5   8      4       2   1   3
        RF             7     6     5   8      2       4   3   1
        SVM            7     5     6   8      3       2   1   4
HIV     KNN            5     6     8   7      1       4   3   2
        NN             6     8     7   5      1       4   3   2
        RF             4     8     7   5      1       3   2   6
        SVM            4     6     7   5      1       8   3   2
Liver   KNN            8     5     4   7      3       2   1   6
        NN             7     2     4   6      1       8   3   5
        RF             8     6     5   7      2       4   3   1
        SVM            7     5     6   8      1       2   3   4
hERG    KNN            6     7     8   5      1       4   2   3
        NN             4     6     8   7      2       5   1   3
        RF             6     7     8   5      1       4   2   3
        SVM            6     8     7   5      1       4   2   3

In [6]:
path_output_auc_rank = os.path.join(PATH_ROOT, 'plot_results', 'roc', 'auc_pivot.xlsx')
df_auc_pivot.to_excel(path_output_auc_rank)

In [7]:
df_auc_rank = pd.DataFrame()

for d in DATASETS:
    # Get mean value by dataset
    indices_by_dataset = df_auc_pivot.index.get_level_values('Dataset') == d
    rank_means = df_auc_pivot[indices_by_dataset].mean(axis=0)
    # Get ranking of the means
    rank_means = rankdata(rank_means, method='min')
    _df = pd.DataFrame([rank_means], columns=AD_alias)
    _df['Dataset'] = d
    df_auc_rank = df_auc_rank.append(_df, ignore_index=True)

df_auc_rank = df_auc_rank.set_index('Dataset')
df_auc_rank = df_auc_rank.astype(int)
# Sort index. Note that upper case goes 1st.
df_auc_rank = df_auc_rank.sort_index()

In [8]:
path_output_auc_rank_mean = os.path.join(PATH_ROOT, 'plot_results', 'roc', 'auc_mean.xlsx')
df_auc_rank.to_excel(path_output_auc_rank_mean)